In [74]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO

In [90]:
save_list=np.load("save_list.npy")
now_list=np.load("now.npy")

In [76]:
txt = input('請輸入你文件的名稱：')

## Open file   把當月份的代號txt  拿出來
doc_name=txt+".txt"
fp = open(doc_name, "r")
line = fp.readline()
month_predict=[]
## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
#    print (line)
    line=line.replace(" ","")
    month_predict.append(line.replace("\n",""))
    line = fp.readline()
fp.close()

請輸入你文件的名稱：911-2


In [77]:
# 抓上市公司股價
def stock_value(datestr):
    #把當月11號的台股資訊 更新
    #datestr = '20200911'
    # 下載股價
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
    # 整理資料，變成表格
    df = pd.read_csv(StringIO(r.text.replace("=", "")), 
                header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    # 整理一些字串：
    df = df.apply(lambda s: pd.to_numeric(s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"), errors='coerce'))
    # 顯示出來
    #df.head()
    return df

In [78]:
#抓上櫃公司股價
def stock_value2(datestr):
    link = 'http://www.tpex.org.tw/web/stock/aftertrading/daily_close_quotes/stk_quote_download.php?l=zh-tw&d='+datestr+'&s=0,asc,0'
    r = requests.get(link)
    r.ok
    lines = r.text.replace('\r', '').split('\n')
    df = pd.read_csv(StringIO("\n".join(lines[3:])), header=None)
    #df.head()
    df.columns = list(map(lambda l: l.replace(' ',''), lines[2].split(',')))
    #df.index = df['代號']
    #df = df.drop(['代號'], axis=1)
    #df.head()
    return df


In [55]:
predict_day='20200911'
now_day="20200930"

In [57]:
df_last_month=stock_value('20200911')
df_now=stock_value('20200930')

In [58]:
df2_last_month=stock_value2("109/09/11")
df2_now=stock_value2('109/09/30')

In [79]:
month_predict

['2882',
 '1303',
 '5519',
 '3045',
 '3529',
 '2881',
 '6165',
 '3443',
 '2107',
 '2883',
 '2834',
 '1605',
 '8299',
 '8928',
 '2439',
 '3323',
 '6147',
 '8938',
 '1216',
 '6005']

In [80]:
df=df_now
for i in range(len(month_predict)):
    a=df[pd.to_numeric(df['證券代號'], errors='coerce') == float(month_predict[i])]
    try:
        b=a.values.tolist()[0]
        print(b[0])
    except:
        pass

2882.0
1303.0
5519.0
3045.0
2881.0
6165.0
3443.0
2107.0
2883.0
2834.0
1605.0
2439.0
1216.0
6005.0


In [81]:
#拿上市收盤價資料
def take_value(df,id):
    take_info=df[pd.to_numeric(df['證券代號'], errors='coerce') == id]
    value=float(take_info["收盤價"].values)
    return value
#拿上櫃收盤價資料
def take_value2(df,id):
    take_info=df[pd.to_numeric(df['代號'], errors='coerce') == id]
    value=float(take_info["收盤"].values)
    return value

In [82]:
#把預測的股票價錢都拿出來
now_list=[]
for i in range(len(month_predict)):
    temp=[]
    index=float(month_predict[i])
    #print(type(index),index)
    try:
        start_value=take_value(df_last_month,index)
        end_value=take_value(df_now,index)
    except:
        start_value=take_value2(df2_last_month,index)
        end_value=take_value2(df2_now,index)
    #print(i,str(index).replace(".0",""),start_value,end_value)
    temp.append(str(index).replace(".0",""))
    temp.append("")
    temp.append(start_value)
    temp.append(end_value)
    now_list.append(temp)

In [83]:
#得到代號 空白   當月11號價格   現在價格
now_list

[['2882', '', 39.9, 38.6],
 ['1303', '', 60.8, 59.5],
 ['5519', '', 15.05, 14.35],
 ['3045', '', 100.5, 96.6],
 ['3529', '', 499.0, 510.0],
 ['2881', '', 43.45, 41.9],
 ['6165', '', 58.1, 71.1],
 ['3443', '', 259.0, 260.0],
 ['2107', '', 21.2, 19.75],
 ['2883', '', 8.77, 8.51],
 ['2834', '', 9.93, 9.54],
 ['1605', '', 16.55, 16.05],
 ['8299', '', 273.0, 264.5],
 ['8928', '', 30.85, 30.15],
 ['2439', '', 152.0, 148.0],
 ['3323', '', 41.8, 42.85],
 ['6147', '', 57.2, 63.4],
 ['8938', '', 38.5, 39.15],
 ['1216', '', 66.0, 62.5],
 ['6005', '', 10.9, 10.75]]

In [84]:
#查詢list in str
#any('1101' in item for item in a)
#查詢list in str
def search_name(name):
    company=[]
    df = pd.read_csv('公司/上市.csv')  
    a=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/上櫃.csv')  
    b=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/興櫃.csv')  
    c=df["有價證券代號及名稱"].tolist()
    for i in a:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in b:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    for i in c:
        if i.__contains__(name) :
            company=i
            #print(i, " is containing")
    return company

In [85]:
#不要亂執行   拿來刪檔案用得
db =  sqlite3.connect('db.sqlite3')
##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicHistory")
db.commit()
db =  sqlite3.connect('db.sqlite3')
##db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
#db.execute("delete from technicHistory")
db.commit()

In [86]:
def change_parameter(save_list):
    final_update=now_day[:4]+"-"+now_day[4:6]+"-"+now_day[6:8]
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date=predict_day[:4]+"-"+predict_day[4:6]+"-"+predict_day[6:8]
    start_price=now_list[i][2]
    over_date="2020-10-10"
    current_price=now_list[i][3]

    now_return=(float(current_price)-float(start_price))/float(start_price)
    now_return= round(now_return*100,2)

    if float(now_return)>0:
        types="+"
    else:
        types="-"
    return final_update,name,start_date,start_price,over_date,current_price,now_return,types

In [87]:
change_parameter(now_list[1])

('2020-09-30', '6005 群益證', '2020-09-11', 10.9, '2020-10-10', 10.75, -1.38, '-')

In [88]:
# 更新月營收策略  當月股價
for i in range(len(now_list)):
    final_update,name,start_date,start_price,over_date,current_price,now_return,types=change_parameter(now_list[i]) 
    print(final_update,name,start_date,start_price,over_date,current_price,now_return,types)

2020-09-30 2882 國泰金 2020-09-11 39.9 2020-10-10 38.6 -3.26 -
2020-09-30 1303 南亞 2020-09-11 60.8 2020-10-10 59.5 -2.14 -
2020-09-30 5519 隆大 2020-09-11 15.05 2020-10-10 14.35 -4.65 -
2020-09-30 3045 台灣大 2020-09-11 100.5 2020-10-10 96.6 -3.88 -
2020-09-30 3529 力旺 2020-09-11 499.0 2020-10-10 510.0 2.2 +
2020-09-30 2881 富邦金 2020-09-11 43.45 2020-10-10 41.9 -3.57 -
2020-09-30 6165 捷泰 2020-09-11 58.1 2020-10-10 71.1 22.38 +
2020-09-30 3443 創意 2020-09-11 259.0 2020-10-10 260.0 0.39 +
2020-09-30 2107 厚生 2020-09-11 21.2 2020-10-10 19.75 -6.84 -
2020-09-30 2883 開發金 2020-09-11 8.77 2020-10-10 8.51 -2.96 -
2020-09-30 2834 臺企銀 2020-09-11 9.93 2020-10-10 9.54 -3.93 -
2020-09-30 1605 華新 2020-09-11 16.55 2020-10-10 16.05 -3.02 -
2020-09-30 8299 群聯 2020-09-11 273.0 2020-10-10 264.5 -3.11 -
2020-09-30 8928 鉅明 2020-09-11 30.85 2020-10-10 30.15 -2.27 -
2020-09-30 2439 美律 2020-09-11 152.0 2020-10-10 148.0 -2.63 -
2020-09-30 3323 加百裕 2020-09-11 41.8 2020-10-10 42.85 2.51 +
2020-09-30 6147 頎邦 2020-09-11 57.2 2

In [89]:
# 更新月營收策略  當月股價
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
pk=1
for i in range(len(now_list)):
    final_update,name,start_date,start_price,over_date,current_price,now_return,types=change_parameter(now_list[i]) 
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [91]:
#基本面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

In [92]:
#技術面歷史
pk=1
for i in range(len(save_list)):
    name=search_name(save_list[i][0]).replace("\u3000", " ")
    start_date=save_list[i][1]
    over_date=save_list[i][2]
    buy_price=save_list[i][3]
    sell_price=save_list[i][4]
    return_value=(float(sell_price)-float(buy_price))/float(buy_price)
    return_value= round(return_value*100,2)
    if return_value>0:
        types="+"
    else:
        types="-"

    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicHistory (id,stock_name,start_date,buy_price,over_date,sell_price,return_value,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,name,start_date,buy_price,over_date,sell_price,return_value,types))
    db.commit()
    db.close()
    pk=pk+1

IntegrityError: UNIQUE constraint failed: technicHistory.id

In [40]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
#db.execute("delete from technicCurrent")
#db.commit()

In [ ]:
db =  sqlite3.connect('db.sqlite3')
#db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
db.execute("delete from basicCurrent")
db.commit()

In [10]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1

In [11]:
#基本面現在
pk=1
for i in range(len(now_list)):
    final_update="2020-07-14"
    name=search_name(now_list[i][0]).replace("\u3000", " ")
    start_date="2020-07-10"
    start_price=now_list[i][2]
    over_date="2020-08-14"
    current_price=now_list[i][3]
    now_return=now_list[i][1]
    if float(now_return)>0:
        types="+"
    else:
        types="-"
        
        
    db =  sqlite3.connect('db.sqlite3')
    db.execute("INSERT INTO technicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
    db.commit()
    db.close()
    pk=pk+1